In [13]:
import warnings
import pandas as pd
from numpy import *
from datetime import timedelta
import yfinance as yf
from tabulate import tabulate

# Math & Optimization
from scipy.stats import norm
from scipy.optimize import fsolve

# Plotting
import matplotlib.pyplot as plt
from PIL import Image
import cufflinks as cf
cf.set_config_file(offline=True)

# Ignore warnings
warnings.filterwarnings('ignore')


In [14]:
class BS:

    """
    This is a class for Options contract for pricing European options on stocks/index without dividends.
    
    Attributes: 
        spot          : int or float
        strike        : int or float 
        rate          : float
        dte           : int or float [days to expiration in number of years]
        volatility    : float
        callprice     : int or float [default None]
        putprice      : int or float [default None]
    """

    def __init__(self, spot, strike, rate, dte, volatility, callprice=None, putprice=None):

        # Spot Price
        self.spot = spot

        # Option Strike
        self.strike = strike

        # Interest Rate
        self.rate = rate

        # Days To Expiration
        self.dte = dte

        # Volatlity
        self.volatility = volatility

        # Callprice # mkt price
        self.callprice = callprice

        # Putprice # mkt price
        self.putprice = putprice

        # Utility
        self._a_ = self.volatility * self.dte**0.5

        if self.strike == 0:
            raise ZeroDivisionError('The strike price cannot be zero')
        else:
            self._d1_ = (log(self.spot / self.strike) +
                         (self.rate + (self.volatility**2) / 2) * self.dte) / self._a_

        self._d2_ = self._d1_ - self._a_

        self._b_ = e**-(self.rate * self.dte)

        # The __dict__ attribute
        '''
        Contains all the attributes defined for the object itself. It maps the attribute name to its value.
        '''
        for i in ['callPrice', 'putPrice', 'callDelta', 'putDelta', 'callTheta', 'putTheta',
                  'callRho', 'putRho', 'vega', 'gamma', 'impvol']:
            self.__dict__[i] = None

        [self.callPrice, self.putPrice] = self._price()
        [self.callDelta, self.putDelta] = self._delta()
        [self.callTheta, self.putTheta] = self._theta()
        [self.callRho, self.putRho] = self._rho()
        self.vega = self._vega()
        self.gamma = self._gamma()
        self.impvol = self._impvol()

    # Option Price
    def _price(self):
        '''Returns the option price: [Call price, Put price]'''

        if self.volatility == 0 or self.dte == 0:
            call = maximum(0.0, self.spot - self.strike)
            put = maximum(0.0, self.strike - self.spot)
        else:
            call = self.spot * norm.cdf(self._d1_) - self.strike * e**(-self.rate *
                                                                       self.dte) * norm.cdf(self._d2_)

            put = self.strike * e**(-self.rate * self.dte) * norm.cdf(-self._d2_) - \
                self.spot * norm.cdf(-self._d1_)
        return [call, put]

    # Option Delta
    def _delta(self):
        '''Returns the option delta: [Call delta, Put delta]'''

        if self.volatility == 0 or self.dte == 0:
            call = 1.0 if self.spot > self.strike else 0.0
            put = -1.0 if self.spot < self.strike else 0.0
        else:
            call = norm.cdf(self._d1_)
            put = -norm.cdf(-self._d1_)
        return [call, put]

    # Option Gamma
    def _gamma(self):
        '''Returns the option gamma'''
        return norm.pdf(self._d1_) / (self.spot * self._a_)

    # Option Vega
    def _vega(self):
        '''Returns the option vega'''
        if self.volatility == 0 or self.dte == 0:
            return 0.0
        else:
            return self.spot * norm.pdf(self._d1_) * self.dte**0.5 / 100

    # Option Theta
    def _theta(self):
        '''Returns the option theta: [Call theta, Put theta]'''
        call = -self.spot * norm.pdf(self._d1_) * self.volatility / (
            2 * self.dte**0.5) - self.rate * self.strike * self._b_ * norm.cdf(self._d2_)

        put = -self.spot * norm.pdf(self._d1_) * self.volatility / (
            2 * self.dte**0.5) + self.rate * self.strike * self._b_ * norm.cdf(-self._d2_)
        return [call / 365, put / 365]

    # Option Rho
    def _rho(self):
        '''Returns the option rho: [Call rho, Put rho]'''
        call = self.strike * self.dte * self._b_ * norm.cdf(self._d2_) / 100
        put = -self.strike * self.dte * self._b_ * norm.cdf(-self._d2_) / 100

        return [call, put]

    # Option Implied Volatility
    def _impvol(self):
        '''Returns the option implied volatility'''
        if (self.callprice or self.putprice) is None:
            return self.volatility
        else:
            def f(sigma):
                option = BS(self.spot, self.strike, self.rate, self.dte, sigma)
                if self.callprice:
                    # f(x) = BS_Call - MarketPrice
                    return option.callPrice - self.callprice
                if self.putprice and not self.callprice:
                    return option.putPrice - self.putprice

            return maximum(1e-5, fsolve(f, 0.2)[0])


In [15]:
option = BS(100, 100, 0.02, 1, 0.2, callprice=10)

header = ['Option Price', 'Delta', 'Gamma', 'Theta', 'Vega', 'Rho', 'IV']
table = [[option.callPrice, option.callDelta, option.gamma,
          option.callTheta, option.vega, option.callRho, option.impvol]]

print(tabulate(around(table, 3), header, tablefmt="grid"))


+----------------+---------+---------+---------+--------+-------+-------+
|   Option Price |   Delta |   Gamma |   Theta |   Vega |   Rho |    IV |
+================+=========+=========+=========+========+=======+=======+
|          8.916 |   0.579 |    0.02 |  -0.013 |  0.391 |  0.49 | 0.228 |
+----------------+---------+---------+---------+--------+-------+-------+


----

In [16]:
def newton_iv(className, spot, strike, rate, dte, volatility, callprice=None, putprice=None):

    x0 = 1  # initial guess
    h = 0.001
    tolerance = 1e-7
    epsilon = 1e-14  # some kind of error or floor

    maxiter = 200

    if callprice:
        # f(x) = Black Scholes Call price - Market Price
        def f(x): return eval(className)(
            spot, strike, rate, dte, x).callPrice - callprice
    if putprice:
        def f(x): return eval(className)(
            spot, strike, rate, dte, x).putPrice - putprice

    for i in range(maxiter):
        y = f(x0)
        yprime = (f(x0+h) - f(x0-h))/(2*h)      # central difference

        if abs(yprime) < epsilon:
            # this is critial, because volatility cannot be negative
            break
        x1 = x0 - y/yprime

        if (abs(x1-x0) <= tolerance*abs(x1)):
            break
        x0 = x1

    return x1


In [17]:
opt = BS(100, 100, 0.02, 1, 0.2)
opt

In [18]:
newton_iv('BS', 100, 100, 0.02, 1, 0.2, callprice=8)


0.17657213831399154

In [19]:
newton_iv('BS', 100, 100, 0.02, 1, 0.2, callprice=8.916037278572539)


0.20000000000000015

In [20]:
newton_iv('BS', 100, 100, 0.02, 1, 0.2, putprice=10)


0.27842040930050715

---

In [21]:
# Bisection Method
def bisection_iv(className, spot, strike, rate, dte, volatility, callprice=None, putprice=None, high=500.0, low=0.0):

    if callprice:
        price = callprice
    if putprice and not callprice:
        price = putprice

    tolerance = 1e-7

    for i in range(1000):
        mid = (high + low) / 2  # c= (a+b)/2
        if mid < tolerance:
            mid = tolerance

        if callprice:
            estimate = eval(className)(spot, strike, rate, dte,
                                       mid).callPrice  # Blackscholes price
        if putprice:
            estimate = eval(className)(spot, strike, rate, dte, mid).putPrice

        if round(estimate, 6) == price:
            break
        elif estimate > price:
            high = mid  # b = c
        elif estimate < price:
            low = mid  # a = c

    return mid


In [22]:
bisection_iv('BS', 100, 100, 0.02, 1, 0.2, callprice=8)


0.17657213902566582

In [23]:
spx  = yf.Ticker('^SPX')

In [25]:
expiry_dates = sorted(spx.options)
expiry_dates[:5]

['2021-11-12', '2021-11-15', '2021-11-17', '2021-11-19', '2021-11-22']

In [26]:
spx_call = spx.option_chain().calls
spx_puts = spx.option_chain().puts

In [28]:
#filtrando as call options que expiry em 2021
sep = spx.option_chain('2021-11-17')

sep = sep.calls[sep.calls['strike']>4535]
sep.set_index('strike', inplace=True)

sep.iloc[:,:11].head()

,contractSymbol,lastTradeDate,lastPrice,bid,ask,change,percentChange,volume,openInterest,impliedVolatility,inTheMoney
strike,,,,,,,,,,,
4540.0,SPXW211117C04540000,2021-11-11 20:04:52,115.50,0.0,0.0,-47.289993,-29.049692,6.0,558,0.00001,True
4545.0,SPXW211117C04545000,2021-11-11 18:28:20,110.90,0.0,0.0,-47.210000,-29.858960,5.0,24,0.00001,True
4550.0,SPXW211117C04550000,2021-11-11 16:23:48,110.05,0.0,0.0,-18.250000,-14.224474,76.0,354,0.00001,True
4555.0,SPXW211117C04555000,2021-11-11 19:55:29,103.10,0.0,0.0,-44.100000,-29.959238,5.0,243,0.00001,True
4560.0,SPXW211117C04560000,2021-11-08 19:23:19,146.55,0.0,0.0,0.000000,0.000000,5.0,761,0.00001,True


In [29]:

sep[['bid', 'impliedVolatility']][:4750].iplot(secondary_y='impliedVolatility', mode='lines+markers', size=6,
                                               xTitle='Strike Price', title='Option Call Price Vs Implied Volatility')


In [31]:
# Spot prices
spx_spot = 4535
rate = 0.007

# Set valuation and expiry dates
valuation_date = pd.Timestamp('2021-09-01')
expiration_date = pd.Timestamp('2021-09-30')
dte = (expiration_date-valuation_date) / \
    timedelta(365)       # time to maturity in years

# Filter dataframe with ltp
df = sep[['bid', 'ask']]
df['mid'] = 0.5 * (df['bid']+df['ask'])                     # (ask+bid)/2

# Add strike column
df['strike'] = df.index

# Initialize the IV column
df['IV'] = 0.

for i in range(len(df)):
    df['IV'].iloc[i] = 100*BS(spx_spot,
                              df.strike.iloc[i],
                              rate, dte, 0.2,
                              callprice=df.mid.iloc[i]).impvol
df.head()

,bid,ask,mid,strike,IV
strike,,,,,
4540.0,0.0,0.0,0.0,4540.0,20.0
4545.0,0.0,0.0,0.0,4545.0,20.0
4550.0,0.0,0.0,0.0,4550.0,20.0
4555.0,0.0,0.0,0.0,4555.0,20.0
4560.0,0.0,0.0,0.0,4560.0,20.0


In [32]:
df['IV'][:4750].iplot(mode='lines+markers', size=6,
                      xTitle='Strike Price', title=' SEP 2021 Call IV')


In [33]:
def ivskew(spot, rate, value_date, expiry, dataframe):

    valuation_date = pd.Timestamp(value_date)
    expiration_date = pd.Timestamp(expiry)
    dte = (expiration_date-valuation_date)/timedelta(365)

    df = dataframe[['bid', 'ask']]
    df['mid'] = 0.5 * (df['bid']+df['ask'])
    df['strike'] = df.index
    df['IV'] = 0.

    for i in range(len(df)):
        df['IV'].iloc[i] = 100*BS(spot, df.strike.iloc[i],
                                  rate, dte, 0.2, callprice=df.mid.iloc[i]).impvol

    return df['IV']


In [35]:
# Set expiry list
expiry_list = ['2021-11-17', '2021-11-30']

# Initialise empty dataframe
call_close = pd.DataFrame()
call_vols = pd.DataFrame()

for expiry in expiry_list:

    sub_set = spx.option_chain(expiry).calls
    sub_set = sub_set[(sub_set.strike > 4535) & (sub_set.strike < 4750)]
    sub_set.set_index('strike', inplace=True)

    call_close[expiry] = sub_set['ask']
    call_vols[expiry] = ivskew(spx_spot, rate, '2021-09-01', expiry, sub_set)


In [36]:
call_vols.iloc[:10]


,2021-11-17,2021-11-30
strike,,
4540.0,20.0,20.0
4545.0,20.0,20.0
4550.0,20.0,20.0
4555.0,20.0,20.0
4560.0,20.0,20.0
4565.0,20.0,20.0
4570.0,20.0,20.0
4575.0,20.0,20.0
4580.0,20.0,20.0


In [37]:
call_close.iplot(mode='lines+markers', symbol=['circle-dot', 'square'], size=6,
                 title='Option Call Price', xTitle='strike price')


In [38]:
call_vols[:4750].iplot(mode='lines+markers', symbol=['circle-dot', 'square'], size=6,
                       title='Expiry-wise IV Skew', xTitle='strike price')
